In [24]:
import math as m
import torch as t
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
from d2l import torch as d2l
device = t.device("cuda")
criterion = nn.MSELoss()
import time
from torch.nn import functional as F

In [43]:
x = t.randn(2,100,requires_grad=True)   
w = t.randn(100,50,requires_grad=True)       
b = t.randn(1,50,requires_grad=True)
#y = t.randn(1,50)
a=1.5

In [44]:
def matrix_fractional_differentiation_linear(x,w,b,a):
    
    x_rows, x_cols = x.size()
    w_rows, w_cols = w.size()
    x = (t.sum(x,dim=0)/x_rows).view(1,-1)
   
    y = F.linear(x,w.T,b)
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    xx = t.abs(x)
    x_partial = t.mul(c_partial.T,xx**(-a)/gamma(1-a)).T
    x_main = t.mul(w.T,xx**(1-a)/gamma(2-a)).T
    
    #w
    ww = t.abs(w)
    w_partial = t.mul(c_partial,ww**(-a)/gamma(1-a))
    w_main = t.mul(x,ww.T**(1-a)/gamma(2-a)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main+w_partial,dim=1)/w_cols).view(1,-1)
    w_matrix_fractional_differentiation = t.repeat_interleave(w_matrix_fractional_differentiation, repeats=x_rows, dim=0) 
    
    #b
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial = t.mul(b_c_partial,bb**(-a)/gamma(1-a))
    b_main = bb**(1-a)/gamma(2-a)
    
    return x_main+x_partial, w_matrix_fractional_differentiation, b_main+b_partial

def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return m.gamma(a)

In [45]:
start = time.time()
s1,s2,s3 = matrix_fractional_differentiation_linear(x,w,b,a)
end = time.time()
print(end - start)

0.0009987354278564453


In [46]:
s1.shape,s2.shape,s3.shape

(torch.Size([100, 50]), torch.Size([2, 100]), torch.Size([1, 50]))

In [89]:
start = time.time()
s1,s2,s3 = matrix_fractional_differentiation_linear1(x,w,b,a)
end = time.time()
print(end - start)

3.108616352081299


In [79]:
def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return m.gamma(a)

#Explicit computation
def matrix_fractional_differentiation_linear1(x,w,b,a):    
    
    #x, w, b = x.to(device), w.to(device), b.to(device)
    #x_rows, x_cols = x.size()
    w_rows, w_cols = w.size()
    #b_rows, b_cols = w.size()
    x_new = t.zeros_like(x)
    w_new = t.zeros_like(w)
    w_new_x = t.zeros_like(w)
    b_new = t.zeros_like(b)
    y = t.mm(x,w) + b   #x_rows*w_cols
    
    #x
    xx = t.abs(x)
    for i in range(w_cols):   #m
        for j in range(w_rows):   #n
            c = y[0,i] - x[0,j]*w[j,i]
            w_new[j,i] = w[j,i]*xx[0,j]**(1-a)/gamma(2-a) + c*xx[0,j]**(-a)/gamma(1-a)
    x_matrix_fractional_differentiation = w_new
    
    # w
    ww = t.abs(w)
    for i in range(w_cols):   #m
        for j in range(w_rows):   #n
            c = y[0,i] - x[0,j]*w[j,i]
            w_new_x[j,i] = x[0,j]*ww[j,i]**(1-a)/gamma(2-a) + c*ww[j,i]**(-a)/gamma(1-a)
    for i in range(w_rows):  #n
        for j in range(w_cols):   #m
            x_new[0,i] = x_new[0,i] + w_new_x[i,j]
        x_new[0,i] = x_new[0,i]/w_cols
    w_matrix_fractional_differentiation = x_new
    
   #b
    bb = t.abs(b)
    for i in range(w_cols):   #m
        c = y[0,i] - b[0,i]
        b_new[0,i] = bb[0,i]**(1-a)/gamma(2-a) + c*bb[0,i]**(-a)/gamma(1-a)
        #This is a piece of code designed to accommodate the initial 'b' coming from nn.Linear.
        #c = y[0,i] - b[i]   
        #b_new[i] = bb[0,i]**(1-a)/gamma(2-a) + c*bb[0,i]**(-a)/gamma(1-a)
    b_matrix_fractional_differentiation = b_new    
    return x_matrix_fractional_differentiation, w_matrix_fractional_differentiation, b_matrix_fractional_differentiation